In [1]:
import pandas as pd
import numpy as np
import time
import datetime

%run "../1. Librerias Mongo/MongoDB_Connections.ipynb"
%run "../1. Librerias Mongo/MongoDB_Funciones_Consultas.ipynb"

### Carga de datos de entrada

In [2]:
# MOVIMIENTOS

#DF = pd.read_csv('../../data/DataFrame_Final_Cierre.csv', parse_dates=['FECHA'])
df_tracks = pd.read_csv('../../data/DataFrame_Final_Cierre_Cluster.csv', parse_dates=['FECHA'])

df_clusterFull = pd.read_csv('../../data/Clusterizada_DataFull.csv')
df_clusterGeog = pd.read_csv('../../data/Clusterizada_soloGeografico.csv')
df_clusterDmnd = pd.read_csv('../../data/Clusterizada_soloDemandaRatio.csv')


In [3]:
df_tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2142248 entries, 0 to 2142247
Data columns (total 24 columns):
 #   Column               Dtype         
---  ------               -----         
 0   ESTACION             int64         
 1   ANIO                 int64         
 2   MES                  int64         
 3   DIA                  int64         
 4   HORA                 int64         
 5   FECHA                datetime64[ns]
 6   DIA_SEMANA           int64         
 7   AM_PM                object        
 8   TEMPORADA            object        
 9   TEMPORADA_NUM        int64         
 10  Es_Festivo           int64         
 11  Es_FinSemana         int64         
 12  TEMPERATURA          float64       
 13  VIENTO               float64       
 14  PRESION              int64         
 15  HUMEDAD              int64         
 16  PRECIPITACION_1h     float64       
 17  PRECIPITACION_3h     float64       
 18  DESC_TIEMPO          object        
 19  DESC_TIEMPO_detalle  

In [4]:
# ESTACIONES: Lista de estaciones en DF

df_estaciones = df_tracks.ESTACION.unique()
df_estaciones

array([  6,   8,   9,  12,  15,  17,  19,  20,  21,  25,  26,  31,  36,
        39,  40,  41,  43,  44,  45,  47,  48,  49,  50,  52,  53,  54,
        55,  56,  57,  58,  59,  63,  65,  66,  67,  69,  72,  73,  74,
        75,  76,  77,  78,  79,  80,  81,  82,  85,  87,  90,  92,  93,
        95,  96,  98, 101, 102, 103, 104, 105, 106, 107, 110, 113, 114,
       122, 123, 124, 125, 126, 128, 129, 130, 131, 132, 133, 134, 135,
       136, 139, 145, 148, 149, 150, 152, 153, 154, 155, 156, 157, 158,
       160, 161, 162, 163, 164, 166, 167, 168, 170, 172, 175,   7,  11,
        13,  14,  16,  18,  27,  34,  51,  64,  71,  83,  91,  94, 118,
       140, 141, 142, 143, 147,   5,  42,  46,  62,  86, 117, 121, 137,
       165,   4, 111, 116, 146, 151,  30,  89, 108, 112, 159, 169,   3,
        60,  88,  97, 109, 115, 120, 138, 171,  33,  84,  99, 127, 144,
       173, 174,   1, 100,  10,  35,  38,   2,  32,  37, 119,  61,  29,
        28, 179, 191, 178, 195, 199, 215, 192, 193, 198], dtype=

In [5]:
# Meteorologia

db_Meteo = _connect_mongo('cloud', 'cluster0.15npsxw.mongodb.net', None, 'ucmtfm2022', 'UCM_2022', 'BiciMAD', 'Meteo_US_NivelHora')

t_ini = time.time()
#data_Meteo = _consulta_meteoUS_por_anio(db_Meteo, 2020)
data_Meteo = _consulta_meteoUS_full(db_Meteo)
t_end = time.time()

print (t_end - t_ini)

# Se agrega campo FECHA para join
data_Meteo['FECHA'] =pd.to_datetime({'year': data_Meteo['ANIO'],
                                          'month': data_Meteo["MES"]
                                          ,'day':  data_Meteo['DIA']
                                          ,'hour':data_Meteo['HORA']},
                                          format='%d-%m-%Y', errors='coerce')

data_Meteo.info()

Conexion OK
Collection(Database(MongoClient(host=['ac-x1d17w2-shard-00-02.15npsxw.mongodb.net:27017', 'ac-x1d17w2-shard-00-01.15npsxw.mongodb.net:27017', 'ac-x1d17w2-shard-00-00.15npsxw.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-xyv6ql-shard-0', tls=True, serverselectiontimeoutms=4000, tlscafile='C:\\ProgramData\\Anaconda3\\lib\\site-packages\\certifi\\cacert.pem'), 'BiciMAD'), 'Meteo_US_NivelHora')
1.0684337615966797
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26304 entries, 0 to 26303
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ANIO                 26304 non-null  int64         
 1   MES                  26304 non-null  int64         
 2   DIA                  26304 non-null  int64         
 3   HORA                 26304 non-null  int64         
 4   TEMPERATURA          26304 non-null  object        
 5 

### Creación de DF de fechas

In [6]:
# FECHAS DE FESTIVOS

df_festivos_in = pd.read_csv('../../data/Festivos_Madrid.csv', sep=';')
# Se agrega campo FECHA para join
df_festivos_in['FECHA'] =pd.to_datetime({'year': df_festivos_in['ANIO'],
                                          'month': df_festivos_in["MES"]
                                          ,'day':  df_festivos_in['DIA']},
                                          format='%d-%m-%Y', errors='coerce')
df_festivos_in.info()
df_festivos = df_festivos_in[df_festivos_in['FESTIVO']==1]

df_festivos.info()
df_festivos[df_festivos['ANIO']==2018].head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461 entries, 0 to 1460
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   ANIO     1461 non-null   int64         
 1   MES      1461 non-null   int64         
 2   DIA      1461 non-null   int64         
 3   FESTIVO  1461 non-null   int64         
 4   FECHA    1461 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(4)
memory usage: 57.2 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 57 entries, 0 to 1454
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   ANIO     57 non-null     int64         
 1   MES      57 non-null     int64         
 2   DIA      57 non-null     int64         
 3   FESTIVO  57 non-null     int64         
 4   FECHA    57 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(4)
memory usage: 2.7 KB


,ANIO,MES,DIA,FESTIVO,FECHA
365,2018,1,1,1,2018-01-01
370,2018,1,6,1,2018-01-06
452,2018,3,29,1,2018-03-29
453,2018,3,30,1,2018-03-30
485,2018,5,1,1,2018-05-01


### Creación de rango total de fechas

In [7]:
# Creación de rangos de fecha válidos
# sdate = datetime.strptime('01/01/2018 00', '%d/%m/%Y %H')
# edate = datetime.strptime('31/01/2018 23', '%d/%m/%Y %H')

sdate = np.datetime64('2018-01-01T00:00:00')
edate = np.datetime64('2020-02-29T23:00:00')

df_dates=pd.date_range(start=sdate, end=edate, freq='H')

print(df_dates.shape)
df_dates

(18960,)


DatetimeIndex(['2018-01-01 00:00:00', '2018-01-01 01:00:00',
               '2018-01-01 02:00:00', '2018-01-01 03:00:00',
               '2018-01-01 04:00:00', '2018-01-01 05:00:00',
               '2018-01-01 06:00:00', '2018-01-01 07:00:00',
               '2018-01-01 08:00:00', '2018-01-01 09:00:00',
               ...
               '2020-02-29 14:00:00', '2020-02-29 15:00:00',
               '2020-02-29 16:00:00', '2020-02-29 17:00:00',
               '2020-02-29 18:00:00', '2020-02-29 19:00:00',
               '2020-02-29 20:00:00', '2020-02-29 21:00:00',
               '2020-02-29 22:00:00', '2020-02-29 23:00:00'],
              dtype='datetime64[ns]', length=18960, freq='H')

### EJECUCION DE PROCESO 

In [8]:
# df_estaciones = [175]
# type(df_estaciones)

In [9]:
clusterFull = df_clusterFull.loc[df_clusterFull['ESTACION'] == df_estaciones[0], ['CLUSTER']].iloc[0]['CLUSTER']
type(clusterFull)

numpy.int64

In [10]:
i=0
df_final = pd.DataFrame(columns = ['ESTACION', 'ANIO', 'MES', 'DIA', 'HORA', 'FECHA', 'DIA_SEMANA', 'AM_PM', 'TEMPORADA', 
                            'TEMPORADA_NUM', 'Es_Festivo', 'Es_FinSemana', 'TEMPERATURA', 'VIENTO', 'PRESION', 'HUMEDAD', 
                            'PRECIPITACION_1h', 'PRECIPITACION_3h', 'DESC_TIEMPO', 'DESC_TIEMPO_detalle', 'CLUSTER_FullCols', 
                            'CLUSTER_soloGeo', 'CLUSTER_soloDemanda', 'DEMANDA'])

t_ini = time.time()

for estacion in df_estaciones:
    print('Estacion: '+str(estacion))
    
    df_data = df_tracks[df_tracks['ESTACION']==estacion]
    
    df_final_estacion = pd.DataFrame(columns = ['ESTACION', 'ANIO', 'MES', 'DIA', 'HORA', 'FECHA', 'DIA_SEMANA', 'AM_PM', 'TEMPORADA', 
                            'TEMPORADA_NUM', 'Es_Festivo', 'Es_FinSemana', 'TEMPERATURA', 'VIENTO', 'PRESION', 'HUMEDAD', 
                            'PRECIPITACION_1h', 'PRECIPITACION_3h', 'DESC_TIEMPO', 'DESC_TIEMPO_detalle', 'CLUSTER_FullCols', 
                            'CLUSTER_soloGeo', 'CLUSTER_soloDemanda', 'DEMANDA'])    
    
    t_ini_estacion = time.time()
    for fecha in df_dates:
        
        # Variable dia semana
        if fecha.isoweekday()==7:
            dia_semana=1
        else:
            dia_semana=fecha.isoweekday()+1
            
        # Variable Es_Festivo
        fec_festivo = np.datetime64(fecha.date())        
        if fec_festivo in df_festivos['FECHA'].values:
            es_festivo = 1
        else:
            es_festivo = 0
                        
        # Variable Es_FinSemana
        if ((dia_semana==1) or (dia_semana==7)):
            es_finsemana = 1
        else:
            es_finsemana = 0
        
        # Variable AM_PM
        if fecha.hour<=12:
            ampm='AM'
        else:
            ampm='PM'
            
        # Variable TEMPORADA
        if fecha.month<=3:
            temporada='INVIERNO'
            temporada_num=1
        elif fecha.month<=6:
            temporada='PRIMAVERA'
            temporada_num=2
        elif fecha.month<=9:
            temporada='VERANO'
            temporada_num=3
        else:
            temporada='OTONO'
            temporada_num=4
            
        clusterFull = df_clusterFull.loc[df_clusterFull['ESTACION'] == estacion, ['CLUSTER']].iloc[0]['CLUSTER']        
        clusterGeog = df_clusterGeog.loc[df_clusterGeog['ESTACION'] == estacion, ['CLUSTER']].iloc[0]['CLUSTER']
        clusterDmnd = df_clusterDmnd.loc[df_clusterDmnd['ESTACION'] == estacion, ['CLUSTER']].iloc[0]['CLUSTER']
                
                
        if fecha not in df_data[df_data['ESTACION']==estacion]['FECHA'].values:
            i=i+1
            meteo = data_Meteo[data_Meteo['FECHA']==fecha]
            new_row_dict = {
                'ESTACION': estacion,
                'ANIO': fecha.year,
                'MES': fecha.month,
                'DIA': fecha.day,
                'HORA': fecha.hour,
                'FECHA': fecha,
                'DIA_SEMANA': dia_semana,
                'AM_PM': ampm,
                'TEMPORADA': temporada,
                'TEMPORADA_NUM': temporada_num,
                'Es_Festivo': es_festivo,
                'Es_FinSemana': es_finsemana,
                'TEMPERATURA': float(meteo.TEMPERATURA.iloc[0]),
                'VIENTO': float(meteo.VIENTO.iloc[0]),
                'PRESION': float(meteo.PRESION.iloc[0]),
                'HUMEDAD': float(meteo.HUMEDAD.iloc[0]),
                'PRECIPITACION_1h': float(meteo.PRECIPITACION_1h.iloc[0]),
                'PRECIPITACION_3h': float(meteo.PRECIPITACION_3h.iloc[0]),
                'DESC_TIEMPO': meteo.DESC_TIEMPO.iloc[0],
                'DESC_TIEMPO_detalle': meteo.DESC_TIEMPO_detalle.iloc[0],
                'CLUSTER_FullCols': clusterFull,
                'CLUSTER_soloGeo': clusterGeog,
                'CLUSTER_soloDemanda': clusterDmnd,
                'DEMANDA': 0
            }
            
            new_row = pd.DataFrame(new_row_dict, index=[0])
            
            #print(meteo.TEMPERATURA.iloc[0])
            df_final_estacion = pd.concat([df_final_estacion, new_row])
            
            #df = pd.concat([df, i])
        else:
            #print(df_data[(df_data['ESTACION']==estacion) & (df_data['FECHA']==fecha)])
            old_row = df_data[(df_data['ESTACION']==estacion) & (df_data['FECHA']==fecha)]
            df_final_estacion = pd.concat([df_final_estacion, old_row])
            
            
    df_final = pd.concat([df_final, df_final_estacion])
    
    t_end_estacion = time.time()
    print('Fechas vacías:' + str(i))
    print('Tiempo: '+str(t_end_estacion-t_ini_estacion))
    i=0
    

t_end = time.time()
print (t_end - t_ini)

Estacion: 6
Fechas vacías:3550
Tiempo: 73.3277108669281
Estacion: 8
Fechas vacías:4590
Tiempo: 73.81846022605896
Estacion: 9
Fechas vacías:4355
Tiempo: 71.75799870491028
Estacion: 12
Fechas vacías:5325
Tiempo: 72.20662546157837
Estacion: 15
Fechas vacías:7629
Tiempo: 71.06495785713196
Estacion: 17
Fechas vacías:4766
Tiempo: 71.91966080665588
Estacion: 19
Fechas vacías:3303
Tiempo: 73.3746280670166
Estacion: 20
Fechas vacías:6376
Tiempo: 71.06476163864136
Estacion: 21
Fechas vacías:6325
Tiempo: 71.54961085319519
Estacion: 25
Fechas vacías:5988
Tiempo: 72.80268979072571
Estacion: 26
Fechas vacías:3599
Tiempo: 78.1555848121643
Estacion: 31
Fechas vacías:4042
Tiempo: 73.55855870246887
Estacion: 36
Fechas vacías:7382
Tiempo: 70.9843807220459
Estacion: 39
Fechas vacías:7650
Tiempo: 71.21905851364136
Estacion: 40
Fechas vacías:7364
Tiempo: 70.99847912788391
Estacion: 41
Fechas vacías:4427
Tiempo: 71.3741147518158
Estacion: 43
Fechas vacías:3269
Tiempo: 72.38821625709534
Estacion: 44
Fechas va

Fechas vacías:3924
Tiempo: 68.52374291419983
Estacion: 3
Fechas vacías:4498
Tiempo: 68.53391242027283
Estacion: 60
Fechas vacías:6811
Tiempo: 67.77686405181885
Estacion: 88
Fechas vacías:11467
Tiempo: 62.60225534439087
Estacion: 97
Fechas vacías:9254
Tiempo: 66.74178957939148
Estacion: 109
Fechas vacías:9299
Tiempo: 66.88597106933594
Estacion: 115
Fechas vacías:6896
Tiempo: 67.71278810501099
Estacion: 120
Fechas vacías:10146
Tiempo: 65.88901543617249
Estacion: 138
Fechas vacías:9337
Tiempo: 66.46347880363464
Estacion: 171
Fechas vacías:6329
Tiempo: 67.74997353553772
Estacion: 33
Fechas vacías:6823
Tiempo: 67.7214629650116
Estacion: 84
Fechas vacías:4867
Tiempo: 68.470139503479
Estacion: 99
Fechas vacías:6330
Tiempo: 67.51846766471863
Estacion: 127
Fechas vacías:8346
Tiempo: 66.56541609764099
Estacion: 144
Fechas vacías:11260
Tiempo: 63.098747968673706
Estacion: 173
Fechas vacías:11034
Tiempo: 63.668753147125244
Estacion: 174
Fechas vacías:8736
Tiempo: 67.3075065612793
Estacion: 1
Fecha

In [11]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3393840 entries, 0 to 2142246
Data columns (total 24 columns):
 #   Column               Dtype         
---  ------               -----         
 0   ESTACION             object        
 1   ANIO                 object        
 2   MES                  object        
 3   DIA                  object        
 4   HORA                 object        
 5   FECHA                datetime64[ns]
 6   DIA_SEMANA           object        
 7   AM_PM                object        
 8   TEMPORADA            object        
 9   TEMPORADA_NUM        object        
 10  Es_Festivo           object        
 11  Es_FinSemana         object        
 12  TEMPERATURA          float64       
 13  VIENTO               float64       
 14  PRESION              object        
 15  HUMEDAD              object        
 16  PRECIPITACION_1h     float64       
 17  PRECIPITACION_3h     float64       
 18  DESC_TIEMPO          object        
 19  DESC_TIEMPO_detalle  

In [12]:
df_final['CLUSTER_FullCols'] = df_final['CLUSTER_FullCols'].astype(np.int64)
df_final['CLUSTER_soloGeo'] = df_final['CLUSTER_soloGeo'].astype(np.int64)
df_final['CLUSTER_soloDemanda'] = df_final['CLUSTER_soloDemanda'].astype(np.int64)
df_final['TEMPORADA_NUM'] = df_final['TEMPORADA_NUM'].astype(np.int64)
df_final['DEMANDA'] = df_final['DEMANDA'].astype(np.int64)

df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3393840 entries, 0 to 2142246
Data columns (total 24 columns):
 #   Column               Dtype         
---  ------               -----         
 0   ESTACION             object        
 1   ANIO                 object        
 2   MES                  object        
 3   DIA                  object        
 4   HORA                 object        
 5   FECHA                datetime64[ns]
 6   DIA_SEMANA           object        
 7   AM_PM                object        
 8   TEMPORADA            object        
 9   TEMPORADA_NUM        int64         
 10  Es_Festivo           object        
 11  Es_FinSemana         object        
 12  TEMPERATURA          float64       
 13  VIENTO               float64       
 14  PRESION              object        
 15  HUMEDAD              object        
 16  PRECIPITACION_1h     float64       
 17  PRECIPITACION_3h     float64       
 18  DESC_TIEMPO          object        
 19  DESC_TIEMPO_detalle  

In [13]:
df_final

,ESTACION,ANIO,MES,DIA,HORA,FECHA,DIA_SEMANA,AM_PM,TEMPORADA,TEMPORADA_NUM,...,PRESION,HUMEDAD,PRECIPITACION_1h,PRECIPITACION_3h,DESC_TIEMPO,DESC_TIEMPO_detalle,CLUSTER_FullCols,CLUSTER_soloGeo,CLUSTER_soloDemanda,DEMANDA
0,6,2018,1,1,0,2018-01-01 00:00:00,2,AM,INVIERNO,1,...,1031,56,0.0,0.0,Clear,sky is clear,0,4,0,2
0,6,2018,1,1,1,2018-01-01 01:00:00,2,AM,INVIERNO,1,...,1031.0,55.0,0.0,0.0,Clear,sky is clear,0,4,0,0
184,6,2018,1,1,2,2018-01-01 02:00:00,2,AM,INVIERNO,1,...,1031,60,0.0,0.0,Clear,sky is clear,0,4,0,2
0,6,2018,1,1,3,2018-01-01 03:00:00,2,AM,INVIERNO,1,...,1030.0,59.0,0.0,0.0,Clear,sky is clear,0,4,0,0
305,6,2018,1,1,4,2018-01-01 04:00:00,2,AM,INVIERNO,1,...,1032,57,0.0,0.0,Clear,sky is clear,0,4,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,198,2020,2,29,19,2020-02-29 19:00:00,7,PM,INVIERNO,1,...,1013.0,60.0,0.0,0.0,Clouds,few clouds,1,2,2,0
0,198,2020,2,29,20,2020-02-29 20:00:00,7,PM,INVIERNO,1,...,1014.0,61.0,0.0,0.0,Clouds,few clouds,1,2,2,0
0,198,2020,2,29,21,2020-02-29 21:00:00,7,PM,INVIERNO,1,...,1014.0,67.0,0.0,0.0,Clouds,few clouds,1,2,2,0
0,198,2020,2,29,22,2020-02-29 22:00:00,7,PM,INVIERNO,1,...,1015.0,69.0,0.0,0.0,Clouds,few clouds,1,2,2,0


In [14]:
df_final[df_final['DEMANDA']==0]

,ESTACION,ANIO,MES,DIA,HORA,FECHA,DIA_SEMANA,AM_PM,TEMPORADA,TEMPORADA_NUM,...,PRESION,HUMEDAD,PRECIPITACION_1h,PRECIPITACION_3h,DESC_TIEMPO,DESC_TIEMPO_detalle,CLUSTER_FullCols,CLUSTER_soloGeo,CLUSTER_soloDemanda,DEMANDA
0,6,2018,1,1,1,2018-01-01 01:00:00,2,AM,INVIERNO,1,...,1031.0,55.0,0.0,0.0,Clear,sky is clear,0,4,0,0
0,6,2018,1,1,3,2018-01-01 03:00:00,2,AM,INVIERNO,1,...,1030.0,59.0,0.0,0.0,Clear,sky is clear,0,4,0,0
0,6,2018,1,1,7,2018-01-01 07:00:00,2,AM,INVIERNO,1,...,1032.0,79.0,0.0,0.0,Clear,sky is clear,0,4,0,0
0,6,2018,1,1,8,2018-01-01 08:00:00,2,AM,INVIERNO,1,...,1032.0,82.0,0.0,0.0,Clear,sky is clear,0,4,0,0
0,6,2018,1,1,9,2018-01-01 09:00:00,2,AM,INVIERNO,1,...,1032.0,84.0,0.0,0.0,Clear,sky is clear,0,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,198,2020,2,29,18,2020-02-29 18:00:00,7,PM,INVIERNO,1,...,1012.0,67.0,0.0,0.0,Clouds,few clouds,1,2,2,0
0,198,2020,2,29,19,2020-02-29 19:00:00,7,PM,INVIERNO,1,...,1013.0,60.0,0.0,0.0,Clouds,few clouds,1,2,2,0
0,198,2020,2,29,20,2020-02-29 20:00:00,7,PM,INVIERNO,1,...,1014.0,61.0,0.0,0.0,Clouds,few clouds,1,2,2,0
0,198,2020,2,29,21,2020-02-29 21:00:00,7,PM,INVIERNO,1,...,1014.0,67.0,0.0,0.0,Clouds,few clouds,1,2,2,0


In [22]:
df_final.groupby('CLUSTER_soloGeo').count()

,ESTACION,ANIO,MES,DIA,HORA,FECHA,DIA_SEMANA,AM_PM,TEMPORADA,TEMPORADA_NUM,...,VIENTO,PRESION,HUMEDAD,PRECIPITACION_1h,PRECIPITACION_3h,DESC_TIEMPO,DESC_TIEMPO_detalle,CLUSTER_FullCols,CLUSTER_soloDemanda,DEMANDA
CLUSTER_soloGeo,,,,,,,,,,,,,,,,,,,,,
0,568800,568800,568800,568800,568800,568800,568800,568800,568800,568800,...,568800,568800,568800,568800,568800,568800,568800,568800,568800,568800
1,853200,853200,853200,853200,853200,853200,853200,853200,853200,853200,...,853200,853200,853200,853200,853200,853200,853200,853200,853200,853200
2,644640,644640,644640,644640,644640,644640,644640,644640,644640,644640,...,644640,644640,644640,644640,644640,644640,644640,644640,644640,644640
3,417120,417120,417120,417120,417120,417120,417120,417120,417120,417120,...,417120,417120,417120,417120,417120,417120,417120,417120,417120,417120
4,910080,910080,910080,910080,910080,910080,910080,910080,910080,910080,...,910080,910080,910080,910080,910080,910080,910080,910080,910080,910080


### Validación de resultados en base a suma de DEMANDA

In [23]:
pd.set_option('display.max_rows', None)

In [24]:
# DF Original
df_tracks.groupby('ESTACION').agg(DEMANDA=('DEMANDA', 'sum'), CUENTA=('DEMANDA', 'count'))
#df_tracks.groupby('ESTACION').agg(DEMANDA=('DEMANDA', 'sum'), CUENTA=('DEMANDA', 'count')).to_clipboard()

,DEMANDA,CUENTA
ESTACION,,
1,63706,13912
2,28020,9586
3,52275,14462
4,35135,11690
5,32312,12880
6,66034,15410
7,39552,13381
8,45164,14370
9,75799,14605


In [25]:
# DF Corregido
df_final.groupby('ESTACION').agg(DEMANDA=('DEMANDA', 'sum'), CUENTA=('DEMANDA', 'count'))
#df_final.groupby('ESTACION').agg(DEMANDA=('DEMANDA', 'sum'), CUENTA=('DEMANDA', 'count')).to_clipboard()

,DEMANDA,CUENTA
ESTACION,,
1,63706,18960
2,28020,18960
3,52275,18960
4,35135,18960
5,32312,18960
6,66034,18960
7,39552,18960
8,45164,18960
9,75799,18960


In [26]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3393840 entries, 0 to 2142246
Data columns (total 24 columns):
 #   Column               Dtype         
---  ------               -----         
 0   ESTACION             object        
 1   ANIO                 object        
 2   MES                  object        
 3   DIA                  object        
 4   HORA                 object        
 5   FECHA                datetime64[ns]
 6   DIA_SEMANA           object        
 7   AM_PM                object        
 8   TEMPORADA            object        
 9   TEMPORADA_NUM        int64         
 10  Es_Festivo           object        
 11  Es_FinSemana         object        
 12  TEMPERATURA          float64       
 13  VIENTO               float64       
 14  PRESION              object        
 15  HUMEDAD              object        
 16  PRECIPITACION_1h     float64       
 17  PRECIPITACION_3h     float64       
 18  DESC_TIEMPO          object        
 19  DESC_TIEMPO_detalle  

In [27]:
pd.set_option('display.max_rows', 10)

### Export df_final a CSV

In [28]:
df_final.to_csv('../../data/DataFrame_Final_Cierre_Cluster_FullFecha.csv', index=False)
#df_final.to_csv('../../data/test.csv', index=False)

In [ ]:
test=df_final.groupby(by='TEMPORADA').count()
test

In [ ]:
test=df_final.groupby(by='ANIO').count()
test

In [ ]:
df_final.groupby('ANIO').agg(DEMANDA=('DEMANDA', 'sum'))

In [ ]:
DF2 = pd.read_csv('../../data/test.csv', parse_dates=['FECHA'])
DF2.info()